In [ ]:
delta_url = "https://delta.xanmarta.workers.dev/0:/"

import os, time, threading
from google.colab import output
import os


def init():
    %run 'DELTA.ipynb'


def read_file(WORKER_NAME, WORKER_STATE):
    global delta_url
    WORK = ""
    PARA = []
    print('----- Downloading necessary files -----')
    try:
        trying = 0
        if not os.path.isfile('/content/rclone.conf'):
            while True:
                link = delta_url + WORKER_NAME + '/' + WORKER_STATE + '/rclone.conf'
                print('Downloading', link)
                !curl -O $link
                with open('/content/rclone.conf', 'r') as f:
                    ct = f.read()
                    if (not '<!DOCTYPE html>' in ct) and (not 'usageLimits' in ct):
                        break
                    !rm '/content/rclone.conf'
                    time.sleep(1)
                    trying += 1
                    if trying == 10:
                        print('Download', link, 'unsuccesfully')
                        return WORK, PARA
        trying = 0
        if not os.path.isfile('/content/path.txt'):
            while True:
                link = delta_url + WORKER_NAME + '/' + WORKER_STATE + '/path.txt'
                print('Downloading', link)
                !curl -O $link
                with open('/content/path.txt') as f:
                    ct = f.read()
                    if (not '<!DOCTYPE html>' in ct) and (not 'usageLimits' in ct):
                        break
                    !rm '/content/path.txt'
                    time.sleep(1)
                    trying += 1
                    if trying == 10:
                        print('Download', link, 'unsuccessfully')
                        return WORK, PARA
        print('----- Completed -----')
    except:
        print('Downloading error')
        return WORK, PARA
    with open('/content/path.txt', 'r') as f:
        content = f.read().split('\n')
        WORK = content.pop(0)
        PARA = content
    return WORK, PARA


def clear_output():
    while True:
        output.clear()
        time.sleep(200)


def worker(WORKER_NAME, WORKER_STATE):
    init()
    if os.path.isdir('/content/sample_data'):
        !rm -r '/content/sample_data'
    WORK, PARA = read_file(WORKER_NAME, WORKER_STATE)
    threading.Thread(target=clear_output).start()
    if WORK == 'DELTA':
        delta(PARA)
